In [4]:
import os, warnings
warnings.filterwarnings('ignore')

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data.', one_hot = True)

Extracting ./mnist/data.\train-images-idx3-ubyte.gz
Extracting ./mnist/data.\train-labels-idx1-ubyte.gz
Extracting ./mnist/data.\t10k-images-idx3-ubyte.gz
Extracting ./mnist/data.\t10k-labels-idx1-ubyte.gz


In [6]:
# 28 * 28 * 1(색 하나)로 구성된 ?만큼의 카드
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

## Conv -(Relu) - Pooling
## Conv -(Relu) - Pooling
## FC

In [30]:
# 3 * 3 * 1의 필터를 32개
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1, 1, 1, 1], padding = 'SAME')
print(L1) # 패딩은 전과 같은 결과가 나오게 하고, X와 W1을 곱해줌

Tensor("Conv2D_7:0", shape=(?, 28, 28, 32), dtype=float32)


In [31]:
L1 = tf.nn.relu(L1) # relu로 바꿔줌
print(L1)

Tensor("Relu_8:0", shape=(?, 28, 28, 32), dtype=float32)


In [52]:
# ?, 28, 28, 32의 28, 28만 2로 해서 크기를 줄여주기
L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
print(L1)

Tensor("MaxPool_9:0", shape=(?, 7, 7, 32), dtype=float32)


In [33]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1, 1, 1, 1], padding = 'SAME')
print(L2)

Tensor("Conv2D_8:0", shape=(?, 14, 14, 64), dtype=float32)


In [34]:
L2 = tf.nn.relu(L2)
print(L2)

Tensor("Relu_9:0", shape=(?, 14, 14, 64), dtype=float32)


In [35]:
L2 = tf.nn.max_pool(L2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],
                   padding = 'SAME')
print(L2)

Tensor("MaxPool_8:0", shape=(?, 7, 7, 64), dtype=float32)


In [40]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev = 0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
print(L3)

Tensor("Reshape_6:0", shape=(?, 3136), dtype=float32)


In [41]:
L3 = tf.matmul(L3, W3)
print(L3)

Tensor("MatMul_6:0", shape=(?, 256), dtype=float32)


In [43]:
L3 = tf.nn.relu(L3)
print(L3)

Tensor("Relu_11:0", shape=(?, 256), dtype=float32)


In [44]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev = 0.01))
model = tf.matmul(L3, W4)

## 비용, 최적화 함수

In [45]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model,
                                                                labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [46]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
total_batch

550

In [53]:
%%time

total_cost = 0
epoch = 0
for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape(-1, 28, 28, 1)
    
    _, cost_val = sess.run([optimizer, cost],
                          feed_dict = {X: batch_xs,
                                       Y: batch_ys})
    
    total_cost += cost_val
    
print('Epoch: {:.4f} \t Avg. cost:{:.3f}'.format(epoch + 1, total_cost / total_batch))
print('최적화 완료')

Epoch: 1.0000 	 Avg. cost:0.087
최적화 완료
Wall time: 5min 9s
